ch05-How to Implement GAN Hacks in Keras to Train Stable Models

- [How to Implement GAN Hacks in Keras to Train Stable Models](https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/), by [Jason Brownlee](https://machinelearningmastery.com/author/jasonb/) on July 12, 2019 in [Generative Adversarial Networks](https://machinelearningmastery.com/category/generative-adversarial-networks/)

- Excerpt by Ivan HP Lin (220509)

Generative Adversarial Networks, or GANs, are challenging to train.

This is because the architecture involves both a **generator** and a **discriminator** model that compete in a **zero-sum game**. It means that improvements to one model come at the cost of a degrading of performance in the other model. The result is a very unstable training process that can often lead to failure, e.g. a *generator that generates the same image all the time or generates nonsense*.

As such, there are a number of heuristics or best practices (called “GAN hacks“) that can be used when configuring and training your GAN models. These heuristics are been hard won by practitioners testing and evaluating hundreds or thousands of combinations of configuration operations on a range of problems over many years.

Some of these heuristics can be challenging to implement, especially for beginners.

Further, some or all of them may be required for a given project, although it may not be clear which subset of heuristics should be adopted, requiring experimentation. This means a practitioner must be ready to implement a given heuristic with little notice.

In this tutorial, you will discover how to implement a suite of best practices or GAN hacks that you can copy-and-paste directly into your GAN project.

After reading this tutorial, you will know:

* The best sources for practical heuristics or hacks when developing generative adversarial networks.
* How to implement seven best practices for the deep convolutional GAN model architecture from scratch.
* How to implement four additional best practices from Soumith Chintala’s GAN Hacks presentation and list.

## 0.. Tutorial Overview
This tutorial is divided into three parts; they are:

1. Heuristics for Training Stable GANs
2. Best Practices for Deep Convolutional GANs
  - Downsample Using Strided Convolutions
  - Upsample Using Strided Convolutions
  - Use LeakyReLU
  - Use Batch Normalization
  - Use Gaussian Weight Initialization
  - Use Adam Stochastic Gradient Descent
  - Scale Images to the Range [-1,1]
3. Soumith Chintala’s GAN Hacks
  - Use a Gaussian Latent Space
  - Separate Batches of Real and Fake Images
  - Use Label Smoothing
  - Use Noisy Labels

## 1.. Best Practices for Deep Convolutional GANs

Perhaps one of the most important steps forward in the design and training of stable GAN models was the 2015 paper by Alec Radford, et al. titled “[Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434).”

In the paper, they describe the Deep Convolutional GAN, or **DCGAN**, approach to GAN development that has become the de facto standard.

We will look at how to implement seven best practices for the DCGAN model architecture in this section.

### 1.1  Downsample Using **Strided Convolution**s

The **discriminator** model is a standard convolutional neural network model that takes an image as input and must output a binary classification as to whether it is real or fake.

It is standard practice with deep convolutional networks to use pooling layers to downsample the input and feature maps with the depth of the network.

This is not recommended for the DCGAN, and instead, they recommend downsampling using [**strided convolutions**](https://machinelearningmastery.com/padding-and-stride-for-convolutional-neural-networks/).

This involves defining a convolutional layer as per normal, but instead of using the default two-dimensional stride of $(1,1)$ to change it to $(2,2)$. This has the effect of downsampling the input, specifically halving the width and height of the input, resulting in output feature maps with one quarter the area.

The example below demonstrates this with a single hidden convolutional layer that uses downsampling strided convolutions by setting the ‘strides‘ argument to (2,2). The effect is the model will downsample the input from 64×64 to 32×32.

In [1]:
# example of downsampling with strided convolutions
from keras.models import Sequential
from keras.layers import Conv2D

# define model
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', input_shape=(64,64,3)))
# summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


1792 == 64 x (3x3x3 + 1)

### 1.2 Upsample Using **Strided Convolutions**

The **generator** model must generate an output image given as input at a random point from the latent space.

The recommended approach for achieving this is to use a **transpose convolutional layer** with a strided convolution. This is a special type of layer that performs the convolution operation in reverse. Intuitively, this means that setting a stride of 2×2 will have the opposite effect, upsampling the input instead of downsampling it in the case of a normal convolutional layer.

By stacking a transpose convolutional layer with strided convolutions, the generator model is able to scale a given input to the desired output dimensions.

The example below demonstrates this with a single hidden transpose convolutional layer that uses upsampling strided convolutions by setting the ‘strides‘ argument to (2,2).

The effect is the model will upsample the input from 64×64 to 128×128.

<figure>
<center>
<img src="https://i.ytimg.com/vi/96_oGE8WyPg/hqdefault.jpg" width="60%">
<figcaption>Transposed Convolution</figcaption>
<img src="https://www.researchgate.net/profile/Shantanu-Shahane/publication/347534508/figure/fig3/AS:971385199407105@1608607687540/Example-of-Transpose-Convolution-with-Stride-2-2-Kernel-size-2-2.ppm" width="90%">
<figcaption>Transposed Convolution (stride=(2x2))</figcaption></center>
</figure>

In [2]:
# example of upsampling with strided convolutions
from keras.models import Sequential
from keras.layers import Conv2DTranspose

# define model
model = Sequential()
model.add(Conv2DTranspose(64, kernel_size=(4,4), strides=(2,2), padding='same', input_shape=(64,64,3)))
# summarize model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose (Conv2DTra  (None, 128, 128, 64)     3136      
 nspose)                                                         
                                                                 
Total params: 3,136
Trainable params: 3,136
Non-trainable params: 0
_________________________________________________________________


3136 = 64 x (4x4x3 + 1)

### 1.3 Use LeakyReLU

The [rectified linear activation unit](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/), or ReLU for short, is a simple calculation that returns the value provided as input directly, or the value 0.0 if the input is 0.0 or less.

The best practice for GANs is to use a variation of the ReLU that allows some values less than zero and learns where the cut-off should be in each node. This is called the **leaky rectified linear activation unit**, or **LeakyReLU** for short.

A negative slope can be specified for the LeakyReLU and the default value of 0.2 is recommended.

Originally, ReLU was recommend for use in the generator model and LeakyReLU was recommended for use in the discriminator model, although more recently, the LeakyReLU is recommended in both models.

The example below demonstrates using the LeakyReLU with the default slope of 0.2 after a convolutional layer in a discriminator model.

<figure>
<center>
<img src="https://deeplearninguniversity.com/wp-content/uploads/2020/06/Screenshot-2020-06-04-at-9.13.01-PM.png" width="50%">
<figcaption>Leaky ReLU function, with α=0.05</figcaption></center>
</figure>


In [3]:
# example of using leakyrelu in a discriminator model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
# define model
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', input_shape=(64,64,3)))
model.add(LeakyReLU(0.2))
# summarize model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


### 1.4 Batch Normalization

<figure>
<center>
<img src="https://i.stack.imgur.com/stBna.png" width="60%">
<figcaption>Batch normalization for image CNN</figcaption></center>
<center>
<img src="https://theaisummer.com/static/ac89fbcf1c115f07ae68af695c28c4a0/ee604/normalization.png" width="60%">
<figcaption>In-layer normalization techniques for training very deep neural networks</figcaption></center></center>
</figure>


<div>
   <figure><center>
      <img src="https://lh6.googleusercontent.com/QT5hSlqurfmHL5ZGnAkWB-fRej7uy3Pryu1n-Rv3WH4ZNYLbKOkpexJf8zpililWAYc3-otGiH9pR-p8sScHAiednLfl4rC9oQUY-0uKdEcmctX6wLLJ2fkA-A6t2w4pukpNrJ8" width="60%">
   </figure>
   <figure><center>
      <img src="https://lh4.googleusercontent.com/b1yEZVxwXMo8Ukf9XvLG37RDjCuTQ0OV6VBuc-Armt8WJzdlrj02Ss5LqYM1Oi5IStk2O9PudtzE41LjGTqmul8KdMQJjo34TMDdU9ncE0chqrMO5PjEL631KDTJRZ6JFVgVxuM" width="60%">
      <figcaption>Batch Normalization by NTU Prof 李宏毅</figcaption></center>
   </figure>
</div>

[Batch normalization](https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/) standardizes the activations from a prior layer to have a **zero mean** and **unit variance**. This has the effect of stabilizing the training process.

Batch normalization is used after the activation of convolution and transpose convolutional layers in the discriminator and generator models respectively.

It is added to the model after the hidden layer, but before the activation, such as LeakyReLU.

The example below demonstrates adding a Batch Normalization layer after a Conv2D layer in a discriminator model but before the activation.

In [1]:
# example of using batch norm in a discriminator model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
# define model
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(LeakyReLU(0.2))
# summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
Total params: 2,048
Trainable params: 1,920
Non-trainable params: 128
_________________________________________________________________


### 1.5 Use Gaussain Weight Initialization

Before a neural network can be trained, the model weights (parameters) must be initialized to small random variables.

The best practice for DCAGAN models reported in the paper is to initialize all weights using a zero-centered Gaussian distribution (the normal or bell-shaped distribution) with a standard deviation of 0.02.

The example below demonstrates defining a random Gaussian weight initializer with a mean of 0 and a standard deviation of 0.02 for use in a transpose convolutional layer in a generator model.

The same weight initializer instance could be used for each layer in a given model.

In [3]:
# example of gaussian weight initialization in a generator model
from keras.models import Sequential
from keras.layers import Conv2DTranspose
from keras.initializers import RandomNormal
# define model
model = Sequential()
init = RandomNormal(mean=0.0, stddev=0.02)
model.add(Conv2DTranspose(64, kernel_size=(4,4), strides=(2,2), \
                          padding='same', kernel_initializer=init, \
                          input_shape=(64,64,3)))
# summarize model
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_transpose_1 (Conv2DT  (None, 128, 128, 64)     3136      
 ranspose)                                                       
                                                                 
Total params: 3,136
Trainable params: 3,136
Non-trainable params: 0
_________________________________________________________________


### 1.6 Adam Stochastic Gradient Descent

Stochastic gradient descent, or SGD for short, is the standard algorithm used to optimize the weights of convolutional neural network models.

There are many variants of the training algorithm. The best practice for training DCGAN models is to use the [**Adam version of stochastic gradient descent**](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) with the learning rate of 0.0002 and the beta1 momentum value of 0.5 instead of the default of 0.9.

The Adam optimization algorithm with this configuration is recommended when both optimizing the discriminator and generator models.

The example below demonstrates configuring the Adam stochastic gradient descent optimization algorithm for training a discriminator model.

In [6]:
# example of using adam when training a discriminator model
from keras.models import Sequential
from keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
# define model
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), strides=(2,2), padding='same', input_shape=(64,64,3)))
# compile model
opt = Adam(learning_rate=0.0002, beta_1=0.5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

### 1.7 Scale Images to the Range $[-1,1]$

It is recommended to use the hyperbolic tangent activation function ($tanh(⊙)$)as the output from the generator model.

As such, it is also recommended that real images used to train the discriminator are [scaled so that their pixel values](https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/) are in the range $[-1,1]$. This is so that the discriminator will always receive images as input, real and fake, that have pixel values in the same range.

Typically, image data is loaded as a NumPy array such that pixel values are 8-bit unsigned integer (uint8) values in the range $[0, 255]$.

First, the array must be converted to floating point values, then rescaled to the required range.

The example below provides a function that will appropriately scale a NumPy array of loaded image data to the required range of $[-1,1]$.

In [7]:
# example of a function for scaling images

# scale image data from [0,255] to [-1,1]
def scale_images(images):
	# convert from unit8 to float32
	images = images.astype('float32')
	# scale from [0,255] to [-1,1]
	images = (images - 127.5) / 127.5
	return images

## 2.. Soumith Chintala’s GAN Hacks  
* [**How to Train a GAN (Youtube)**](https://www.youtube.com/watch?v=X1mUN6dD8uE) - Highly recommended

The video is available on [YouTube](https://www.youtube.com/watch?v=X1mUN6dD8uE) and is highly recommended. A summary of the tips is also available as a GitHub repository titled “[How to Train a GAN? Tips and tricks to make GANs work](https://github.com/soumith/ganhacks)”


In this section, we will review how to implement four additional GAN best practices not covered in the previous section.

### 2.1 Use a Gaussian Latent Space

The latent space defines the shape and distribution of the input to the generator model used to generate new images.

The **DCGAN** recommends sampling from a uniform distribution, meaning that the shape of the latent space is a hypercube.

The more recent best practice is to sample from a standard [Gaussian distribution](https://machinelearningmastery.com/statistical-data-distributions/), meaning that the shape of the latent space is a hypersphere, with a mean of zero and a standard deviation of one.

The example below demonstrates how to generate 500 [random Gaussian points](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/) from a 100-dimensional latent space that can be used as input to a generator model; each point could be used to generate an image.

In [6]:
import numpy as np
# example of sampling from a gaussian latent space
from numpy.random import randn

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape((n_samples, latent_dim))
	return x_input

# size of latent space
n_dim = 100
# number of samples to generate
n_samples = 500
# generate samples
samples = generate_latent_points(n_dim, n_samples)
# summarize
print(f'sample shape={samples.shape}, \nmean={np.round(samples.mean(),3)}, \nvar={np.round(samples.std(),3)}')

sample shape=(500, 100), 
mean=-0.005, 
var=0.998


### 2.2 Separate Batches of Real and Fake Images

The discriminator model is trained using stochastic gradient descent with mini-batches.

* The best practice is to update the discriminator with separate batches of real and fake images rather than combining real and fake images into a single batch.

* This can be achieved by updating the model weights for the discriminator model with two separate calls to the **train_on_batch()** function.

The code snippet below demonstrates how you can do this within the inner loop of code when training your discriminator model.
```
...
# get randomly selected 'real' samples
X_real, y_real = ...
# update discriminator model weights
discriminator.train_on_batch(X_real, y_real)

# generate 'fake' examples
X_fake, y_fake = ...
# update discriminator model weights
discriminator.train_on_batch(X_fake, y_fake)
```

### 2.3 Use Label Smoothing

It is common to use the class label 1 to represent real images and class label 0 to represent fake images when training the discriminator model.

These are called hard labels, as the label values are precise or crisp.

It is a good practice to use soft labels, such as values slightly more or less than 1.0 or slightly more than 0.0 for real and fake images respectively, where the variation for each image is random.

This is often referred to as **label smoothing** and can have a [**regularizing effect**](https://machinelearningmastery.com/introduction-to-regularization-to-reduce-overfitting-and-improve-generalization-error/) when training the model.

The example below demonstrates defining 1,000 labels for the positive class (class=1) and smoothing the label values uniformly into the range [0.7,1.2] as recommended.

In [10]:
# example of positive label smoothing
import numpy as np
from numpy import ones
from numpy.random import random

# example of smoothing class=1 to [0.7, 1.2]
def smooth_positive_labels(y):
	return y - 0.3 + (random(y.shape) * 0.5)


# generate 'real' class labels (1)
n_samples = 1000
y = ones((n_samples, 1))
# smooth labels
y = smooth_positive_labels(y)
# summarize smooth labels
print(f'y.shape={y.shape}, \n(y.min, y.max)=({np.round(y.min(),3)}, {np.round(y.max(),3)})')

y.shape=(1000, 1), 
(y.min, y.max)=(0.701, 1.199)


There have been some suggestions that only positive-class label smoothing is required and to values less than 1.0. Nevertheless, you can also smooth negative class labels.

The example below demonstrates generating 1,000 labels for the negative class (class=0) and smoothing the label values uniformly into the range $[0.0, 0.3]$ as recommended.

In [12]:
# example of negative label smoothing
import numpy as np
from numpy import zeros
from numpy.random import random

# example of smoothing class=0 to [0.0, 0.3]
def smooth_negative_labels(y):
	return y + random(y.shape) * 0.3

# generate 'fake' class labels (0)
n_samples = 1000
y = zeros((n_samples, 1))
# smooth labels
y = smooth_negative_labels(y)
# summarize smooth labels
#print(y.shape, y.min(), y.max())
print(f'y.shape={y.shape}, \n(y.min, y.max)=({np.round(y.min(),3)}, {np.round(y.max(),3)})')

y.shape=(1000, 1), 
(y.min, y.max)=(0.0, 0.299)


### 2.4 Use Noisy Labels

The labels used when training the discriminator model are always correct.

This means that fake images are always labeled with class 0 and real images are always labeled with class 1.

It is recommended to introduce some errors to these labels where some fake images are marked as real, and some real images are marked as fake.

If you are using separate batches to update the discriminator for real and fake images, this may mean randomly adding some fake images to the batch of real images, or randomly adding some real images to the batch of fake images.

If you are updating the discriminator with a combined batch of real and fake images, then this may involve randomly flipping the labels on some images.

The example below demonstrates this by creating 1,000 samples of real (class=1) labels and flipping them with a 5% probability, then doing the same with 1,000 samples of fake (class=0) labels.

In [16]:
# example of noisy labels
import numpy as np
from numpy import ones
from numpy import zeros
from numpy.random import choice

# randomly flip some labels
def noisy_labels(y, prob_flip):
	# determine the number of labels to flip
	n_select = int(prob_flip * y.shape[0])
	# choose labels to flip
	flip_ix = choice([i for i in range(y.shape[0])], size=n_select)
	# invert the labels in place
	y[flip_ix] = 1 - y[flip_ix]
	return y

# generate 'real' class labels (1)
n_samples = 1000
y = ones((n_samples, 1))
# flip labels with 5% probability
y = noisy_labels(y, 0.05)
# summarize labels
print(f'real image y.sum()={y.sum()}')

# generate 'fake' class labels (0)
y = zeros((n_samples, 1))
# flip labels with 5% probability
y = noisy_labels(y, 0.05)
# summarize labels
print(f'fake image y.sum()={y.sum()}')

real image y.sum()=950.0
fake image y.sum()=50.0


## 3.. Further Readings

Papers
* [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks, 2015](https://arxiv.org/abs/1511.06434)
* [Tutorial: Generative Adversarial Networks, NIPS, 2016](https://arxiv.org/abs/1701.00160)
* [Improved Techniques for Training GANs, 2016](https://arxiv.org/abs/1606.03498)

Articles
* [ganhacks: How to Train a GAN? Tips and tricks to make GANs work](https://github.com/soumith/ganhacks)
* [Ian Goodfellow, Introduction to GANs, NIPS 2016](https://www.youtube.com/watch?v=9JpdAg6uMXs)
* [Soumith Chintala, How to train a GAN, NIPS 2016 Workshop on Adversarial Training](https://www.youtube.com/watch?v=X1mUN6dD8uE)